In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/machine-trainslation")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/machine-trainslation


In [1]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv(f"EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
52886,"[2, 565, 496, 878, 459, 304, 1243, 224, 3, 0, ...","[2, 1502, 895, 14287, 224, 3, 0, 0, 0, 0, 0, 0..."
26639,"[2, 270, 574, 701, 1044, 224, 3, 0, 0, 0, 0, 0...","[2, 850, 316, 886, 2164, 224, 3, 0, 0, 0, 0, 0..."
108873,"[2, 270, 1411, 2316, 1666, 521, 4413, 1145, 22...","[2, 3044, 7022, 1614, 1094, 322, 1571, 2130, 2..."
6923,"[2, 270, 384, 5561, 224, 3, 0, 0, 0, 0, 0, 0, ...","[2, 1963, 22524, 224, 3, 0, 0, 0, 0, 0, 0, 0, ..."
122532,"[2, 658, 304, 607, 429, 272, 700, 304, 3587, 9...","[2, 453, 302, 4841, 2032, 1379, 952, 2219, 704..."
...,...,...
70310,"[2, 479, 2299, 334, 924, 481, 1633, 224, 3, 0,...","[2, 352, 2256, 318, 3038, 292, 12727, 224, 3, ..."
14257,"[2, 341, 304, 1352, 2898, 224, 3, 0, 0, 0, 0, ...","[2, 927, 1685, 224, 3, 0, 0, 0, 0, 0, 0, 0, 0,..."
105280,"[2, 270, 1603, 370, 778, 904, 696, 356, 498, 3...","[2, 328, 1355, 316, 500, 1295, 8411, 669, 960,..."
124816,"[2, 290, 2133, 333, 714, 473, 805, 369, 1408, ...","[2, 290, 1778, 316, 302, 1062, 879, 369, 2417,..."


In [2]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
52886,"[2, 565, 496, 878, 459, 304, 1243, 224, 3, 0, ...","[2, 1502, 895, 14287, 224, 3, 0, 0, 0, 0, 0, 0..."
26639,"[2, 270, 574, 701, 1044, 224, 3, 0, 0, 0, 0, 0...","[2, 850, 316, 886, 2164, 224, 3, 0, 0, 0, 0, 0..."
108873,"[2, 270, 1411, 2316, 1666, 521, 4413, 1145, 22...","[2, 3044, 7022, 1614, 1094, 322, 1571, 2130, 2..."
6923,"[2, 270, 384, 5561, 224, 3, 0, 0, 0, 0, 0, 0, ...","[2, 1963, 22524, 224, 3, 0, 0, 0, 0, 0, 0, 0, ..."
122532,"[2, 658, 304, 607, 429, 272, 700, 304, 3587, 9...","[2, 453, 302, 4841, 2032, 1379, 952, 2219, 704..."
...,...,...
70310,"[2, 479, 2299, 334, 924, 481, 1633, 224, 3, 0,...","[2, 352, 2256, 318, 3038, 292, 12727, 224, 3, ..."
14257,"[2, 341, 304, 1352, 2898, 224, 3, 0, 0, 0, 0, ...","[2, 927, 1685, 224, 3, 0, 0, 0, 0, 0, 0, 0, 0,..."
105280,"[2, 270, 1603, 370, 778, 904, 696, 356, 498, 3...","[2, 328, 1355, 316, 500, 1295, 8411, 669, 960,..."
124816,"[2, 290, 2133, 333, 714, 473, 805, 369, 1408, ...","[2, 290, 1778, 316, 302, 1062, 879, 369, 2417,..."


In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test_data = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train_data, batch_size=512, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=512, shuffle=False, drop_last=True)

In [4]:
import json

with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [5]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        return output, hidden, cell

In [6]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size

        self.W1 = nn.Linear(hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, enc_out):
        hidden = hidden.unsqueeze(1)
        score = self.v(torch.tanh(self.W1(hidden) + self.W2(enc_out)))
        attention_weights = torch.softmax(score, dim=1)
        context_vector = torch.sum(attention_weights * enc_out, dim=1)
        return context_vector, attention_weights

In [7]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.lstm = nn.LSTM(
            embedding_dim + hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell, encoder_output):
        attention_vector, _ = self.attention(hidden[0], encoder_output)
        x = self.embedding(x)
        x = torch.cat((attention_vector, x), dim=-1)
        x = x.unsqueeze(1)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        output = self.dropout(output)
        output = self.fc(output)
        return output, hidden, cell

In [8]:
import torch.nn as nn
import torch
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, vocab_size, teacher_forcing_ratio=0.5):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.vocab_size = vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def forward(self, src, trg):
        enc_out, hidden, cell = self.encoder(src)
        outputs = torch.zeros(trg.shape[0], trg.shape[1], self.decoder.vocab_size).to("cuda")
        x = trg[:, 0]
        for t in range(1, trg.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell, enc_out)
            output = output.squeeze(1)
            outputs[:, t, :] = output
            use_teacher_forcing = random.random() < self.teacher_forcing_ratio
            x = trg[:, t] if use_teacher_forcing else output.argmax(dim=1)

        return outputs
    
    def predict(self, src):
        enc_out, hidden, cell = self.encoder(src)
        outputs = torch.zeros(src.shape[0], src.shape[1], self.decoder.vocab_size).to("cuda")
        x = src[:, 0]
        for i in range(1, src.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell, enc_out)
            output = output.squeeze(1)
            outputs[:, i, :] = output
            x = output.argmax(dim=1)
        
        return outputs

In [10]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train(
        model,
        train_loader,
        test_loader,
        optimizer,
        criterion,
        device,
        scheduler, 
        num_epochs,
    ):
    writer = SummaryWriter()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        model.encoder.train()
        model.decoder.train()
        epoch_loss = 0
        progress_bar = tqdm(train_loader, desc="Training", leave=False)

        for src, trg in progress_bar:
            src, trg = src.to(device), trg.to(device)
            optimizer.zero_grad()
            outputs = model(src, trg)
            loss = criterion(
                    outputs[:, 1:].reshape(-1, model.vocab_size), trg[:, 1:].reshape(-1)
                )
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}"
                )

        train_loss = epoch_loss / len(train_loader)

        # ---------------- VALIDATION ---------------- #
        model.encoder.eval()
        model.decoder.eval()
        val_epoch_loss = 0
        progress_bar = tqdm(test_loader, desc="Validating", leave=False)

        with torch.no_grad():
            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)
                outputs = model(src, trg)
                loss = criterion(
                        outputs[:, 1:].reshape(-1, model.vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                val_epoch_loss += loss.item()

                progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}"
                    )

        val_loss = val_epoch_loss / len(test_loader)
        scheduler.step(val_loss)

        # Log metrics to TensorBoard
        writer.add_scalar("Loss/Train", train_loss, epoch)
        writer.add_scalar("Loss/Validation", val_loss, epoch)
        writer.add_scalar("Learning Rate", scheduler.get_last_lr()[0], epoch)

        print(
            f"Train loss : {train_loss}, Val_loss : {val_loss}, lr: {scheduler.get_last_lr()[0]}"
        )

    writer.close()
    return train_loss, val_loss

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(seq2seq.parameters(), lr=0.005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2
)

In [12]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

train(
    seq2seq,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    scheduler,
    60,
)


Epoch 1/60


Train loss : 5.956495409745436, Val_loss : 5.446685629231589, lr: 0.005

Epoch 2/60


Train loss : 5.40478697067652, Val_loss : 5.156662906919207, lr: 0.005

Epoch 3/60


Train loss : 5.142882342216296, Val_loss : 4.903921144349234, lr: 0.005

Epoch 4/60


Train loss : 4.882353501442151, Val_loss : 4.6033885564122885, lr: 0.005

Epoch 5/60


...

Train loss : 2.2711946359047523, Val_loss : 2.731559838567461, lr: 0.00015625

Epoch 58/60


Train loss : 2.262163845698039, Val_loss : 2.7061460997377123, lr: 7.8125e-05

Epoch 59/60


Train loss : 2.2593330254921544, Val_loss : 2.705182126590184, lr: 7.8125e-05

Epoch 60/60


Train loss : 2.255899355961726, Val_loss : 2.7238571516105106, lr: 7.8125e-05


(2.255899355961726, 2.7238571516105106)

In [13]:
torch.save(seq2seq.state_dict(), "seq2seq.pth")
torch.save(seq2seq.encoder.state_dict(), "encoder.pth")
torch.save(seq2seq.decoder.state_dict(), "decoder.pth")

# Evaluation

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
seq2seq.load_state_dict(torch.load("seq2seq.pth", weights_only=True))

<All keys matched successfully>

In [15]:
seq2seq.eval()
# with torch.no_grad():
a, b = next(iter(test_loader))
a = a.to(device)
b = b.to(device)
with torch.no_grad():
    x = seq2seq.predict(a)

x = x.argmax(dim=2)
x.shape

torch.Size([512, 18])

In [16]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer(
    f"EnglishOrSpanish/vocab.json", f"EnglishOrSpanish/merges.txt"
)

In [17]:
x = x.cpu().numpy()
b = b.cpu().numpy()
a = a.cpu().numpy()
preds = []
targets = []
original = []
for i in range(x.shape[0]):
    preds.append(tokenizer.decode(x[i]))
    targets.append(tokenizer.decode(b[i]))
    original.append(tokenizer.decode(a[i]))

In [18]:
import pandas as pd

df = pd.DataFrame(
    {"Original": original[:10], "Prediction": preds[:10], "Target": targets[:10]}
)
df.to_csv("attention-unidir.csv", index=False)
df

,Original,Prediction,Target
0,<SOS> i should read the book <EOS><PAD><PAD><P...,<PAD> debería leer el libro <EOS><EOS><EOS><EO...,<SOS> debería leer el libro <EOS><PAD><PAD><PA...
1,<SOS> please drop me off at the station <EOS><...,<PAD> dime por la estación por favor <EOS><EOS...,<SOS> déjeme en la estación por favor <EOS><PA...
2,<SOS> tom doesnt remember doing what everybody...,<PAD> tom no se de lo lo que todo lo que <EOS>...,<SOS> tom no recuerda haber hecho lo que todos...
3,<SOS> we dont care what he does <EOS><PAD><PAD...,<PAD> no te importa lo que él <EOS><EOS><EOS><...,<SOS> no nos importa lo que él haga <EOS><PAD>...
4,<SOS> tell the truth <EOS><PAD><PAD><PAD><PAD>...,<PAD> dile la verdad <EOS><EOS><EOS><EOS><EOS>...,<SOS> decí la verdad <EOS><PAD><PAD><PAD><PAD>...
5,<SOS> tom wont be able to understand any of th...,<PAD> tom no podrá entender nada de esto <EOS>...,<SOS> tom no va a poder entender nada de esto ...
6,<SOS> i seldom go out on monday <EOS><PAD><PAD...,<PAD> normalmente voy a el lunes <EOS><EOS><EO...,<SOS> raramente salgo los lunes <EOS><PAD><PAD...
7,<SOS> i didnt want to disturb the patients <EO...,<PAD> no quería escuchar a los los <EOS><EOS><...,<SOS> no quería molestar a los pacientes <EOS>...
8,<SOS> i can be your best friend or your worst ...,<PAD> puedo ser tu mejor amigo o tu peor enemi...,<SOS> puedo ser tu mejor amigo o tu peor enemi...
9,<SOS> having finished it he went to bed <EOS><...,<PAD> se terminado de que se a a la cama <EOS>...,<SOS> después de haberla terminado se fue a la...


In [ ]:
import sacrebleu


def calculate_bleu_score(preds, targets):
    bleu = sacrebleu.corpus_bleu(preds, [targets], force=True)
    return bleu.score


calculate_bleu_score(preds, targets)

23.85645814060703

In [13]:
import rouge

rouge = rouge.Rouge()
scores = rouge.get_scores(preds, targets, avg=True)
scores

{'rouge-1': {'r': 0.4421783117885481,
  'p': 0.4773645787805937,
  'f': 0.45679683824475587},
 'rouge-2': {'r': 0.2552410046550675,
  'p': 0.2618616444702154,
  'f': 0.2578349746190426},
 'rouge-l': {'r': 0.4353404854438858,
  'p': 0.46961805555555464,
  'f': 0.44958920418139353}}

In [14]:
def ter(preds, targets):
    return sacrebleu.corpus_ter(preds, [targets]).score


ter(preds, targets)

58.77256317689531